In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from data_plugin_for_julio import get_bigb_data, get_benchmark_cluster_data, get_bank_etf_cluster_data

bigb_df = get_bigb_data()[0]
bank_etf_cluster_df = get_bank_etf_cluster_data()
benchmark_cluster_df = get_benchmark_cluster_data()

In [9]:
bigb_df

,open,high,low,close,adjclose,volume,actual_returns
date,,,,,,,
2003-04-21,100.825001,101.158335,98.689999,99.275833,64.360238,5.579605e+06,NaN
2003-04-22,98.722499,102.411667,98.139999,102.307500,66.318708,8.367843e+06,0.030538
2003-04-23,102.270832,103.690000,101.524169,103.319999,66.945530,8.174293e+06,0.009897
2003-04-24,102.003335,102.538332,100.073335,100.773333,65.307020,6.983018e+06,-0.024648
2003-04-25,100.783335,101.002499,98.405831,98.890000,64.106814,6.061628e+06,-0.018689
...,...,...,...,...,...,...,...
2023-04-05,106.646667,107.665000,106.344999,107.210000,107.210000,1.533580e+07,-0.005427
2023-04-06,106.985001,108.005002,106.708336,107.610000,107.610000,1.509477e+07,0.003731
2023-04-10,107.341666,108.524997,106.998333,108.323332,108.323332,1.602218e+07,0.006629


# BIGB Only

In [10]:
# Remove missing values, replace infinite values, and fill NaN with mean
bigb_df = bigb_df.replace([np.inf, -np.inf], np.nan).fillna(bigb_df.mean())

# Create a list of k values and calculate inertia for each k
inertia = []
k = list(range(1, 11))
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(bigb_df)
    inertia.append(model.inertia_)

# Create elbow plot
bigb_elbow_data = {
    'k': k,
    'inertia': inertia
}
bigb_elbow_data_df = pd.DataFrame(bigb_elbow_data)
bigb_elbow_plot = bigb_elbow_data_df.hvplot.line(
    x='k',
    y='inertia',
    title='BIGB Elbow Curve',
    xticks=k
)

print(bigb_elbow_data_df)
print(inertia)
bigb_elbow_plot = bigb_elbow_plot.opts(width=600, height=400, show_grid=True)

# Save the elbow plot as a PNG file in the "images" directory
file_path = 'images/bigb_elbow_plot.png'
hvplot.save(bigb_elbow_plot, file_path, fmt='png')
bigb_elbow_plot


    k       inertia
0   1  3.975479e+18
1   2  1.551805e+18
2   3  7.779098e+17
3   4  4.788585e+17
4   5  3.274916e+17
5   6  2.236848e+17
6   7  1.620954e+17
7   8  1.260429e+17
8   9  9.408013e+16
9  10  7.591304e+16
[3.9754794027358106e+18, 1.5518049340134067e+18, 7.779098103624872e+17, 4.788584725655836e+17, 3.2749157581507366e+17, 2.2368476041858413e+17, 1.6209539955672538e+17, 1.2604291040377514e+17, 9.4080125705521e+16, 7.591303677709077e+16]


:Curve   [k]   (inertia)

# Benchmark Cluster

In [11]:
# Concatenate BIGB and Benchmark DataFrames into a single DataFrame
benchmark_cluster = pd.concat([bigb_cluster_df, benchmark_cluster_df])

# Preprocess the data
benchmark_cluster = benchmark_cluster.replace([np.inf, -np.inf], np.nan).fillna(benchmark_cluster.mean())

# Apply the KMeans algorithm to find clusters
# Choose the optimal number of clusters 
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
benchmark_cluster['cluster'] = kmeans.fit_predict(benchmark_cluster[['annual_return', 'annual_volatility']])

import hvplot.pandas

# Create a dictionary to map cluster numbers to their labels
cluster_labels = {
    0: 'BIGB',
    1: 'Benchmark'
}

# Map cluster numbers to their labels
benchmark_cluster['cluster_label'] = benchmark_cluster['cluster'].map(cluster_labels)

# Create a scatter plot using hvplot
benchmark_scatter_plot = benchmark_cluster.hvplot.scatter(
    x='annual_return',
    y='annual_volatility',
    by='cluster_label',
    cmap='Category10',
    legend='top_left',
    title='Benchmark',
    xlabel='Annual Return',
    ylabel='Annual Volatility',
    alpha=0.7,
    size=100,
    hover=True
)
file_path = 'images/benchmark_scatter_plot.png'
hvplot.save(benchmark_scatter_plot, file_path, fmt='png')
benchmark_scatter_plot.opts(width=800, height=500, show_grid=True)

:NdOverlay   [cluster_label]
   :Scatter   [annual_return]   (annual_volatility)

# Bank ETF Cluster

In [12]:
# Concatenate BIGB and Benchmark DataFrames into a single DataFrame
bank_etf_cluster = pd.concat([bigb_cluster_df, bank_etf_cluster_df])

# Preprocess the data
bank_etf_cluster = bank_etf_cluster.replace([np.inf, -np.inf], np.nan).fillna(bank_etf_cluster.mean())

# Apply the KMeans algorithm to find clusters
# Choose the optimal number of clusters
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
bank_etf_cluster['cluster'] = kmeans.fit_predict(bank_etf_cluster[['annual_return', 'annual_volatility']])

import hvplot.pandas

# Create a dictionary to map cluster numbers to their labels
cluster_labels = {
    0: 'BIGB',
    1: 'Bank_ETF'
}

# Map cluster numbers to their labels
bank_etf_cluster['cluster_label'] = bank_etf_cluster['cluster'].map(cluster_labels)

# Create a scatter plot using hvplot
bank_etf_scatter_plot = bank_etf_cluster.hvplot.scatter(
    x='annual_return',
    y='annual_volatility',
    by='cluster_label',
    cmap='Category10',
    legend='top_left',
    title='Bank ETF',
    xlabel='Annual Return',
    ylabel='Annual Volatility',
    alpha=0.7,
    size=100,
    hover=True
)
file_path = 'images/bank_etf_scatter_plot.png'
hvplot.save(bank_etf_scatter_plot, file_path, fmt='png')
bank_etf_scatter_plot.opts(width=800, height=500, show_grid=True)

:NdOverlay   [cluster_label]
   :Scatter   [annual_return]   (annual_volatility)